In [7]:
!pip install fast-bert
!pip install transformers
!pip install tokenizers

     |████████████████████████████████| 2.9MB 5.8MB/s 
ERROR: fast-bert 1.9.4 has requirement tokenizers==0.8.1.rc1, but you'll have tokenizers 0.9.4 which is incompatible.
  Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1


In [24]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import spacy
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from google.colab import drive
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy
import logging
# Language model Databunch
from fast_bert.data_lm import BertLMDataBunch
# Language model learner
from fast_bert.learner_lm import BertLMLearner

from pathlib import Path
from box import Box

In [2]:
from fast_bert.data_cls import BertDataBunch

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
ROOT_PATH = "./gdrive/My Drive/NLP_Final/Colab"
DATA_PATH = f"{ROOT_PATH}/data"
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Load the data

In [ ]:
data = pd.read_csv(f"{ROOT_PATH}/Misinformation_Data_v2.csv")

In [14]:
X_train, X_test, y_train, y_test = train_test_split(data.text.to_numpy(), data.label.to_numpy(), test_size=0.30, random_state=RANDOM_SEED)

In [ ]:
train_data = {"text": X_train, "label": y_train}
df = pd.DataFrame(train_data) 
df.to_csv(f"{DATA_PATH}/train.csv", index=False)

In [ ]:
test_data = {"text": X_test, "label": y_test}
val_df = pd.DataFrame(test_data) 
val_df.to_csv(f"{DATA_PATH}/val.csv", index=False)

In [21]:
databunch = BertDataBunch(DATA_PATH, ROOT_PATH,
                          tokenizer='bert-base-uncased',
                          train_file='Misinformation_Data_v1.csv',
                          label_file='labels.csv',
                          text_col='text',
                          label_col='label',
                          batch_size_per_gpu=16,
                          max_seq_length=512,
                          multi_gpu=True,
                          multi_label=False,
                          model_type='bert')

In [25]:
logger = logging.getLogger()
device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

learner = BertLearner.from_pretrained_model(
						databunch,
						pretrained_path='bert-base-uncased',
						metrics=metrics,
						device=device_cuda,
						logger=logger,
						output_dir=ROOT_PATH,
						finetuned_wgts_path=None,
						warmup_steps=500,
						multi_gpu=True,
						is_fp16=True,
						multi_label=False,
						logging_steps=50)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
learner.lr_find(start_lr=1e-5,optimizer_type='lamb')

/usr/local/lib/python3.6/dist-packages/pytorch_lamb/lamb.py:96: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)


In [ ]:
learner.fit(epochs=6,
			lr=6e-5,
			validate=True,
			schedule_type="warmup_cosine",
			optimizer_type="lamb")

In [ ]:
learner.save_model()